## **Simulation Set-Up**

In [10]:
import os
import importlib
import sys

import numpy as np
from pydrake.all import (
    AddDefaultVisualization,
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    LoadModelDirectives,
    LoadModelDirectivesFromString,
    Parser,
    ProcessModelDirectives,
    RigidTransform,
    RollPitchYaw,
    Simulator,
    StartMeshcat,
    LeafSystem,
    ConstantVectorSource
)
from IPython.display import display, SVG
import pydot
import matplotlib.pyplot as plt
from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
from pydrake.systems.controllers import JointStiffnessController
from pydrake.visualization import ModelVisualizer

from manipulation import running_as_notebook
from manipulation.station import LoadScenario, MakeHardwareStation
from manipulation.utils import ConfigureParser
from manipulation.systems import AddIiwaDifferentialIK

# sys.path.append('.')
# import env_ingredient_add
# importlib.reload(env_ingredient_add)
import PizzaBot

In [3]:
# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

# Simulation Set Up 

Generating a useful simulation scene is the first step in successful manipulation projects. In this exercise, we will learn how to add the necessary objects to set-up a scene.

**These are the learning goals of this exercise:**
1. Be able to work with objects written in robot description formats such as URDFs and SDFs.
2. Understand how to view and edit collision geometries
3. Understand how to add objects to Drake to create a useful manipulation scene.

A great resource is [Authoring a Multibody Simulation tutorial](https://deepnote.com/workspace/Drake-0b3b2c53-a7ad-441b-80f8-bf8350752305/project/Tutorials-2b4fc509-aef2-417d-a40d-6071dfed9199/notebook/authoring_multibody_simulation-add293478aac40a984845aa3705eefdd?). Parts of this notebook were taken from it.

## Simulated Kitchen Setup & Robot Configuration
In the included `objects/environment_setup.yaml` file, we instantiate all the parts of the kitchen scene.

In [15]:
class PoseTrajectorySource(LeafSystem):
    def __init__(self, pose_trajectory):
        LeafSystem.__init__(self)
        self._pose_trajectory = pose_trajectory
        self.DeclareAbstractOutputPort(
            "pose", lambda: AbstractValue.Make(RigidTransform()), self.CalcPose
        )

    def CalcPose(self, context, output):
        output.set_value(self._pose_trajectory.GetPose(context.get_time()))

def preview_scene_given_directives(model_directive):
    meshcat.Delete()
    scenario = LoadScenario(data=model_directive)
    station = MakeHardwareStation(
        scenario, meshcat, package_xmls=xmls
    )
    simulator = Simulator(station)
    context = simulator.get_mutable_context()
    x0 = station.GetOutputPort("mobile_iiwa.state_estimated").Eval(context)
    station.GetInputPort("mobile_iiwa.desired_state").FixValue(context, x0)
    meshcat.StartRecording()
    simulator.AdvanceTo(2.0 if running_as_notebook else 0.1)
    meshcat.PublishRecording()



In [17]:
if os.getcwd() == "/datasets/_deepnote_work/manipulation/project": 
    scene = open("/work/manipulation/project/objects/environment_setup.yaml")
    # scene = env_ingredient_add.get_environment_set_up()
    xmls = [os.getcwd() + "/package.xml", "/work/manipulation/project/package.xml"]
else:
    scene = open("objects/environment_setup.yaml") # local setup
    # scene = env_ingredient_add.get_environment_set_up()
    xmls = [os.getcwd() + "/package.xml"]

scenario = LoadScenario(data=scene)
bot = PizzaBot(scenario)
bot.print_diagram()
# bot.run_simulation(start_time=2.0)

# preview_scene_given_directives(scene)

In [7]:
# bot.CreateDefaultContext()
# bot.print_diagram()
# bot.plant.num_positions()



In [6]:
# !pwd

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=92fbdad2-a4a9-4ef8-b493-c307060b34a1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>